In [ ]:
import os
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    matthews_corrcoef, f1_score, precision_score, recall_score,
    accuracy_score, roc_auc_score, average_precision_score
)

from pyod.models.ae1svm import AE1SVM
from pyod.models.devnet import DevNet
from pyod.models.lunar import LUNAR
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.iforest import IForest
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.deep_svdd import DeepSVDD
from pyod.models.hbos import HBOS
from pyod.models.pca import PCA
from pyod.models.sod import SOD
from pyod.models.cof import COF
from pyod.models.loda import LODA
from pyod.models.alad import ALAD
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE
from pyod.models.so_gaal import SO_GAAL  # đúng: "sogaal" (không phải "so_gaal")
from pyod.models.mo_gaal import MO_GAAL  # đúng: "mogaal" (không phải "mo_gaal")


models_list = [
    # "CBLOF",
    "KNN",
    "IForest",
    "OCSVM",
    "LOF",
    "DeepSVDD",
    "HBOS",
    "PCA",
    "LODA",
    "ECOD",
    "COPOD",
    "AutoEncoder",
    "DevNet",
    "LUNAR",
    "AE1SVM",
    "ALAD",
    "VAE"
    "SO_GAAL",
    "MO_GAAL"
]

output_file = f"Results/Baseline_OC.csv"
columns = ["Dataset","Model",  "scaled", "noise_percentage", "AUCROC", "AUCPR", "Accuracy", "MCC", "F1 Score",
           "Precision", "Recall", "Time Train", "Time Test"]

def preprocess_data_noise(train_data, test_data, noise_percentage=10):
    """
    Preprocess the training and testing data by separating features and labels,
    and return the preprocessed feature sets and labels with added noise.
    
    Args:
        train_data (DataFrame): Training data containing features and labels.
        test_data (DataFrame): Testing data containing features and labels.
        noise_percentage (float): The percentage of training samples to add noise to.
        
    Returns:
        X_train (ndarray): Preprocessed training features with added noise.
        y_train (ndarray): Preprocessed training labels with added noise.
        X_test (ndarray): Preprocessed testing features.
        y_test (ndarray): Preprocessed testing labels.
    """
    print("..............................Data Overview................................")
    print("Train Data Shape:", train_data.shape)
    print("Test Data Shape:", test_data.shape)
    
    # Convert to numpy arrays for easier manipulation
    X_train_total = train_data.iloc[:, :-1].to_numpy()
    y_train_total = train_data.iloc[:, -1].to_numpy()

    # Separate the samples with label 0
    X_train = X_train_total[y_train_total == 0]
    y_train = y_train_total[y_train_total == 0]

    print("Train Data Labels [0]:", np.unique(y_train))

    # Calculate how many samples to add noise to based on the provided percentage
    n_samples = X_train.shape[0]
    noise_samples_count = int(n_samples * (noise_percentage / 100))

    # Get the samples with label 1 (for generating noise)
    X_train_noise = X_train_total[y_train_total == 1]
    
    # Randomly select noise_samples_count from X_train_noise
    noisy_indices = np.random.choice(X_train_noise.shape[0], size=noise_samples_count, replace=False)
    X_train_noise = X_train_noise[noisy_indices]
    
    # Add the noisy samples to the training set
    X_train = np.vstack((X_train, X_train_noise))
    # y_train = np.concatenate((y_train, np.ones(X_train_noise.shape[0])))
    y_train = np.concatenate((y_train, np.zeros(X_train_noise.shape[0])))

    print(y_train) 
    # Prepare test data
    X_test = test_data.iloc[:, :-1].to_numpy()
    y_test = test_data.iloc[:, -1].to_numpy()

    # Print the new size of training data
    n_samples = X_train.shape[0]
    n_features = X_train.shape[1]
    print("Number of samples after adding noise:", n_samples)
    print("Number of features:", n_features)

    return X_train, y_train, X_test, y_test


def evaluate_model(y_true, y_pred, y_scores=None, y_probabilities=None):
    """
    Evaluates the model using multiple metrics and prints the results.
    This version includes AUCROC and AUCPR using predicted probabilities.

    Parameters:
    - y_true (np.ndarray): True labels of the test data.
    - y_pred (np.ndarray): Predicted labels of the test data.
    - y_scores (np.ndarray, optional): Scores used to compute AUCROC and AUCPR.
    - y_probabilities (np.ndarray, optional): Predicted probabilities for each class.

    Returns:
    - metrics (list): A list containing AUCROC, AUCPR, accuracy, MCC, F1 score, precision, and recall.
    """
    print("..............................Evaluation Metrics...............................")

    # Calculate standard metrics
    mcc = matthews_corrcoef(y_true, y_pred)  # Matthew's correlation coefficient
    f1 = f1_score(y_true, y_pred)  # F1 score
    precision = precision_score(y_true, y_pred)  # Precision
    recall = recall_score(y_true, y_pred)  # Recall
    accuracy = accuracy_score(y_true, y_pred)  # Accuracy

    # ROC and PR curve scores using predicted probabilities
    auc_roc, auc_pr = None, None
    if y_probabilities is not None:
        auc_roc = roc_auc_score(y_true, y_probabilities[:, 1])  # Probabilities for class 1 (outliers)
        auc_pr = average_precision_score(y_true, y_probabilities[:, 1])  # AUCPR for class 1

    # Display metrics
    print(f"AUCROC: {auc_roc * 100 if auc_roc else 'N/A'}")
    print(f"AUCPR: {auc_pr * 100 if auc_pr else 'N/A'}")
    print(f"Accuracy: {accuracy * 100:.2f}")
    print(f"MCC: {mcc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

    # Return the evaluation metrics as a list
    return [auc_roc * 100 if auc_roc else None, auc_pr * 100 if auc_pr else None,
            accuracy * 100, mcc, f1, precision, recall]


def get_model(name, **kwargs):
    """
    Returns the appropriate PyOD model based on the provided name.
    
    Parameters:
    - name (str): The name of the anomaly detection model (e.g., 'CBLOF', 'KNN').
    - kwargs (dict): Additional parameters to initialize the model.

    Returns:
    - model (object): An instance of the specified anomaly detection model.
    """
    model_dict = {
        "CBLOF": CBLOF,
        "KNN": KNN,
        "IForest": IForest,
        "OCSVM": OCSVM,
        "LOF": LOF,
        "DeepSVDD": DeepSVDD,
        "HBOS": HBOS,
        "SOD": SOD,
        "COF": COF,
        "LODA": LODA,
        "PCA": PCA,
        "ECOD": ECOD,
        "COPOD": COPOD,
        "AutoEncoder": AutoEncoder,
        "DevNet": DevNet,
        "LUNAR": LUNAR,
        "AE1SVM": AE1SVM,
        "ALAD": ALAD,
        "VAE": VAE,
        "SO_GAAL": SO_GAAL,
        "MO_GAAL": MO_GAAL
    }
    
    # Fetch the model class based on the provided model name
    model_class = model_dict.get(name)
    
    # Raise an error if the model name is not found
    if model_class is None:
        raise ValueError(f"Model {name} not found.")
    
    # Return the model initialized with the provided parameters
    return model_class(**kwargs)


# Initialize output CSV file
if not os.path.exists(output_file):
    pd.DataFrame(columns=columns).to_csv(output_file, index=False)

# Define a function to run the model training and evaluation
def run_experiment(X_train, y_train, X_test, y_test, name, noise_percentage, scaler ):
    for model_name in models_list:
        try:
            print(f"\nRunning dataset {name} with model {model_name}")
            
            # train_data = pd.concat([X_train, y_train], axis=1, ignore_index=True)
            # test_data = pd.concat([X_test, y_test], axis=1, ignore_index=True)

            # # Preprocess data (assuming preprocess_data is defined elsewhere)
            # X_train, y_train, X_test, y_test = preprocess_data(train_data, test_data)
            
            if model_name == 'DeepSVDD':
                start_time = time.time()
                n_features = X_train.shape[1]
                model = DeepSVDD(n_features=n_features)
                model.fit(X_train)
            else:
                model = get_model(model_name)
                start_time = time.time()
                if model_name == 'DevNet':
                    model.fit(X_train, y_train)
                else:
                    model.fit(X_train)
            train_time = time.time() - start_time

            # Test the model
            start_time = time.time()
            y_pred = model.predict(X_test)
            # Check if the model supports predict_proba and calculate probabilities
            if hasattr(model, "predict_proba"):
                y_probabilities = model.predict_proba(X_test)  # Class probabilities for each instance
            else:
                y_probabilities = None  # Some models do not have predict_proba method

            test_time = time.time() - start_time

            # Evaluate the model using the appropriate probabilities
            metrics = evaluate_model(y_test, y_pred, y_scores=None, y_probabilities=y_probabilities)
            result = [name, model_name] + [scaler, noise] + metrics + [train_time, test_time]
            result_df = pd.DataFrame([result], columns=columns)
            result_df.to_csv(output_file, mode='a', header=False, index=False)

            print(f"Results saved for {name} with model {model_name}")

        except Exception as e:
            print(f"Error with dataset {name}, model {model_name}: {e}")



if __name__ == "__main__": 
    
    # dataset_prefixes =  ['N_BaIoT_dataloader.csv']
    dataset_prefixes =  ['data_ToNIoT.csv']
    # dataset_prefixes =  ['data_N_BaIoT.csv' , 'data_CICIoT2023.csv', 'data_BoTIoT.csv']
    
    # scaler_names = ['MinMaxScaler']
    scaler_names = ['StandardScaler','QuantileTransformer','MinMaxScaler','Normalizer','RobustScaler']
    
    # scaler_names = ['QuantileTransformer']
    
    for prefix in dataset_prefixes:
        
        for scaler in scaler_names: 
         
            # Construct file paths for train and test datasets with 'Train_' and 'Test_' prefixes
            train_file = f'Datascaled/NoiseOCData/Train_{scaler}_{prefix}'
            test_file = f'Datascaled/NoiseOCData/Test_{scaler}_{prefix}'
            
            # Load the CSV files
            df_train = pd.read_csv(train_file)
            df_test = pd.read_csv(test_file)

            
            df_train = df_train.dropna() 
            df_test = df_test.dropna() 
            
            # Nối lại thành 1 DataFrame
            df_full = pd.concat([df_train, df_test], ignore_index=True)
            
            # Chia theo tỉ lệ 70% train, 30% test
            df_train_new, df_test_new = train_test_split(df_full, test_size=0.3, random_state=42)

            # print(f"Số lượng hàng có NaN: {num_rows_with_nan}")

            for noise in range (0 ,1 ): 
                
                # Step 1: Preprocess data
                X_train, y_train, X_test, y_test = preprocess_data_noise(df_train_new, df_test_new, noise)
                # Run the experiments for each dataset type
                run_experiment(X_train, y_train, X_test, y_test, prefix, noise, scaler ) 
               



..............................Data Overview................................
Train Data Shape: (34149, 29)
Test Data Shape: (14636, 29)
Train Data Labels [0]: [0]
[0. 0. 0. ... 0. 0. 0.]
Number of samples after adding noise: 3888
Number of features: 28

Running dataset data_ToNIoT.csv with model KNN
..............................Evaluation Metrics...............................
AUCROC: 85.65412872924063
AUCPR: 95.95181266406507
Accuracy: 54.15
MCC: 0.2492
F1 Score: 0.6559
Precision: 0.9713
Recall: 0.4951
Results saved for data_ToNIoT.csv with model KNN

Running dataset data_ToNIoT.csv with model IForest
..............................Evaluation Metrics...............................
AUCROC: 56.58718889688535
AUCPR: 88.61991468867463
Accuracy: 21.17
MCC: 0.0179
F1 Score: 0.2124
Precision: 0.8984
Recall: 0.1205
Results saved for data_ToNIoT.csv with model IForest

Running dataset data_ToNIoT.csv with model OCSVM
..............................Evaluation Metrics..............................

Training:  10%|█         | 1/10 [00:01<00:12,  1.41s/it]

Epoch 33/100, Loss: 112.971200466156
Epoch 34/100, Loss: 113.7819275856018
Epoch 35/100, Loss: 113.01587349176407
Epoch 36/100, Loss: 112.99517011642456
Epoch 37/100, Loss: 112.90603977441788
Epoch 38/100, Loss: 112.89560920000076
Epoch 39/100, Loss: 113.07099413871765
Epoch 40/100, Loss: 112.89300203323364
